# Predicting Home Sale Price

This will attempt to be similar to the Zestimate used on Zillow, the Redfin estimate used on Redfin, or any of the valuations provided by Collateral Analytics, CoreLogic, or Quantarium on Realtor.com.

I will create a regression model and use it to predict valuation and compare to the other valuations on currently listed homes.

In [116]:
%run src/imports.py

In [2]:
sns.set(rc={'figure.figsize':(16,4)})

In [64]:
neighborhoods_cleaned = pd.read_pickle('data/neighborhoods_cleaned.pkl')
neighborhoods_cleaned.reset_index(inplace=True)
neighborhoods_cleaned.drop(['index','DAYS ON MARKET','PROPERTY TYPE','ADDRESS','CITY','STATE OR PROVINCE'],axis=1,inplace=True)
#neighborhoods_cleaned = neighborhoods_cleaned.merge(pd.get_dummies(neighborhoods_cleaned['LOCATION'],drop_first=True), how='outer',left_index=True,right_index=True)
neighborhoods_cleaned.drop(['LOCATION'],axis=1,inplace=True)
neighborhoods_cleaned['SOLD DATE'] = pd.to_datetime(neighborhoods_cleaned['SOLD DATE'])
neighborhoods_cleaned['MONTH'] = pd.DatetimeIndex(neighborhoods_cleaned['SOLD DATE']).month
neighborhoods_cleaned['YEAR'] = pd.DatetimeIndex(neighborhoods_cleaned['SOLD DATE']).year
neighborhoods_cleaned.drop(['SOLD DATE'],axis=1,inplace=True)

In [65]:
neighborhoods_cleaned

,ZIP OR POSTAL CODE,PRICE,LOT SIZE,YEAR BUILT,$/SQUARE FEET,HOA/MONTH,LATITUDE,LONGITUDE,ROOMS PER SQFT * 1000,MONTH,YEAR
0,78257,345000,5662.0,2005.0,173.0,230.0,29.647801,-98.614186,2.011061,9,2020
1,78256,337500,9583.0,2004.0,156.0,54.0,29.650304,-98.629082,2.770083,6,2020
2,78256,375500,6605.0,2004.0,126.0,54.0,29.651842,-98.630650,2.189289,3,2021
3,78255,462000,9147.0,2014.0,170.0,95.0,29.617650,-98.643430,2.577320,10,2018
4,78256,624000,75794.0,2000.0,207.0,25.0,29.620646,-98.622295,1.826029,2,2021
...,...,...,...,...,...,...,...,...,...,...,...
20007,78222,153000,6969.0,1983.0,97.0,NaN,29.383410,-98.376520,3.170577,9,2019
20008,78222,189900,4791.0,2018.0,112.0,25.0,29.386700,-98.379900,3.523194,7,2018
20009,78222,128000,5662.0,1985.0,116.0,NaN,29.383092,-98.376562,4.537205,3,2019
20010,78222,196499,5662.0,2014.0,85.0,6.0,29.379346,-98.370269,3.012048,3,2019


In [66]:
neighborhoods_cleaned.dropna(inplace=True)

In [67]:
y = neighborhoods_cleaned['PRICE'].values
X = neighborhoods_cleaned.drop('PRICE',axis=1).values

In [68]:
neighborhoods_cleaned

,ZIP OR POSTAL CODE,PRICE,LOT SIZE,YEAR BUILT,$/SQUARE FEET,HOA/MONTH,LATITUDE,LONGITUDE,ROOMS PER SQFT * 1000,MONTH,YEAR
0,78257,345000,5662.0,2005.0,173.0,230.0,29.647801,-98.614186,2.011061,9,2020
1,78256,337500,9583.0,2004.0,156.0,54.0,29.650304,-98.629082,2.770083,6,2020
2,78256,375500,6605.0,2004.0,126.0,54.0,29.651842,-98.630650,2.189289,3,2021
3,78255,462000,9147.0,2014.0,170.0,95.0,29.617650,-98.643430,2.577320,10,2018
4,78256,624000,75794.0,2000.0,207.0,25.0,29.620646,-98.622295,1.826029,2,2021
...,...,...,...,...,...,...,...,...,...,...,...
20002,78222,137000,5227.0,2005.0,49.0,6.0,29.379318,-98.373976,2.163722,6,2018
20003,78222,165000,8276.0,2014.0,79.0,7.0,29.388351,-98.370269,3.117506,1,2020
20006,78222,165900,5662.0,2018.0,133.0,7.0,29.374355,-98.385532,4.815409,2,2019
20008,78222,189900,4791.0,2018.0,112.0,25.0,29.386700,-98.379900,3.523194,7,2018


-----
Data Density  
= 11,654 / 185  
= 62.7
-----

## Building a Model

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [70]:
rfr = RandomForestRegressor()
rfr.fit(X_train,y_train)
y_predict = rfr.predict(X_test)
print("score:", rfr.score(X_test, y_test))
mean_squared_error(y_test, y_predict)

score: 0.9031548267635894


1052095673.0105727

In [71]:
y_predict

array([507329.92, 226049.86, 219557.21, ..., 270831.88, 161264.  ,
       189024.89])

In [72]:
y_test

array([572500, 209900, 275000, ..., 265000, 162500, 190000])

In [94]:
search_space = [{'regressor__alpha': 10 ** (np.linspace(-5, 5, 10)), 
                 'regressor__l1_ratio': np.linspace(0,1,6)}]

# l_lasso / (l_lasso + l_ridge)
# if l_lasso = 0: 0 / 1 = 0 (this is ridge regression)
# if l_ridge = 0: l_lasso / l_lasso = 1 (this is LASSSO regression)

pipe = Pipeline([('scaler', StandardScaler()), ('regressor', ElasticNet())])

enet_cv = GridSearchCV(pipe, 
                      search_space, 
                      cv=KFold(10, shuffle=True), 
                      scoring='neg_mean_squared_error',
                      return_train_score = True)

In [95]:
enet_cv.fit(X_train,y_train)

enet_cv.best_params_

/Users/randallbloomquist/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 18298192065864.008, tolerance: 10381918551.804132
  model = cd_fast.enet_coordinate_descent(
/Users/randallbloomquist/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 17873261824885.523, tolerance: 10138232116.824657
  model = cd_fast.enet_coordinate_descent(
/Users/randallbloomquist/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 16656992441780.867, tolerance: 10358434301.164389
  model = cd_fast.enet_coordinate_descent(
/Users/randallbloomquist/opt/an

{'regressor__alpha': 7742.636826811277, 'regressor__l1_ratio': 0.0}

In [96]:
y_predict = enet_cv.predict(X_test)
mean_squared_error(y_test, y_predict)

10868114492.4807

In [100]:
ridge = Ridge(alpha=1)
ridge.fit(X_train,y_train)
y_predict = ridge.predict(X_test)
print("score:", ridge.score(X_test, y_test))
mean_squared_error(y_test, y_predict)

score: 0.6343582433749522


3972217686.8652315

In [98]:
X_train.shape

(8740, 10)

In [63]:
y_train.shape

(8740,)

In [101]:
lr = LinearRegression()
lr.fit(X_train,y_train)
y_predict = lr.predict(X_test)
print("score:", lr.score(X_test, y_test))
mean_squared_error(y_test, y_predict)

score: 0.6343369635677419


3972448864.03535

In [103]:
np.arange(-5,5,1)

array([-5, -4, -3, -2, -1,  0,  1,  2,  3,  4])

In [110]:
type(y_train)

numpy.ndarray

In [132]:
kf = KFold()
skf = StratifiedKFold()
ridge_pipe = Pipeline([('scaler', StandardScaler()), ('regressor', Ridge())])
ridge_grid_params = {'regressor__alpha': np.arange(0,50,1)}
ridge_grid = GridSearchCV(ridge_pipe, ridge_grid_params, cv=kf, scoring = 'neg_mean_squared_error')
ridge_grid.fit(X_train, y_train)
print("Ridge Accuracy: {:.2e}".format(ridge_grid.score(X_test, y_test)))

KNN Accuracy: -3.97e+09


In [136]:
kf = KFold()
skf = StratifiedKFold()
lasso_pipe = Pipeline([('scaler', StandardScaler()), ('regressor', Lasso())])
lasso_grid_params = {'regressor__alpha': np.arange(0.1,0.9,0.1)}
lasso_grid = GridSearchCV(lasso_pipe, lasso_grid_params, cv=kf, scoring = 'neg_mean_squared_error')
lasso_grid.fit(X_train, y_train)
print("Lasso Accuracy: {:.2e}".format(lasso_grid.score(X_test, y_test)))

Lasso Accuracy: -3.97e+09


In [139]:
kf = KFold()
skf = StratifiedKFold()
gbr_pipe = Pipeline([('scaler', StandardScaler()), ('regressor', GradientBoostingRegressor(loss='huber'))])
gbr_grid_params = {'regressor__alpha': np.arange(0.1,0.9,0.1),
                  'regressor__max_depth': np.arange(1,9,2),
                  'regressor__learning_rate': np.arange(0.01,0.2,0.05)}
gbr_grid = GridSearchCV(gbr_pipe, gbr_grid_params, cv=kf, scoring = 'neg_mean_squared_error')
gbr_grid.fit(X_train, y_train)
print("gbr Accuracy: {:.2e}".format(gbr_grid.score(X_test, y_test)))

gbr Accuracy: -1.03e+09


In [140]:
kf = KFold()
skf = StratifiedKFold()
gbr_pipe2 = Pipeline([('scaler', StandardScaler()), ('regressor', GradientBoostingRegressor(loss='huber'))])
gbr_grid_params2 = {'regressor__alpha': np.arange(0.1,0.9,0.3)}
gbr_grid = GridSearchCV(gbr_pipe, gbr_grid_params, cv=kf, scoring = 'neg_mean_squared_error')
gbr_grid.fit(X_train, y_train)
print("gbr Accuracy: {:.2e}".format(gbr_grid.score(X_test, y_test)))

gbr Accuracy: -1.49e+09


In [118]:
SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted'])